# Seleção de atributos
Neste notebook, selecionaremos os atributos para treinar os modelos de aprendizado de máquina. Como não sabemos a importância de cada atributo de nosso dataset, vamos realizar uma seleção sequencial de atributos movida por um modelo k-NN. Este algoritmo é um algoritmo guloso, portanto, existe a chance de haver perda de dados durante o processo.

In [23]:
#Importações
import pandas as pd
import numpy as np
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
#Importando os dados
df_features_train = pd.read_csv("Features para trino.csv", sep=",")
#Removi a primeira coluna que continha os índices dos participantes, não será necessária.
df_feature_selection = df_features_train.drop("Unnamed: 0", axis=1)

df_target = pd.read_csv("Target para treino.csv", sep=",")
df_features_test = pd.read_csv("Features para teste.csv", sep=",")
df_features_test = df_features_test.drop("Unnamed: 0", axis=1)

#O target está em formato categórico, precisamos transformá-lo em numérico.
y = df_target['Class']           
le = LabelEncoder()
y_enc = le.fit_transform(y)

In [ ]:
# Seleção de atributos:
# Utilizaremos a seleção de atributos por Seleção Sequencial de Atributos. Como métrica, usaremos a média do RMSE de uma cross-val 5-Fold
modelo = KNeighborsClassifier()
seletor = SequentialFeatureSelector(
    modelo,
    n_features_to_select="auto",
    direction="backward",
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)
seletor.fit(df_feature_selection, y_enc)
features_treino = seletor.transform(df_feature_selection)
features_teste = seletor.transform(df_features_test)

print(f"Número original de atributos: {seletor.n_features_in_}")
print(f"Número de atributos selecionados: {seletor.n_features_to_select_}")
print(f"Atributos selecionados: {seletor.get_feature_names_out()}")

Número original de atributos: 134
Número de atributos selecionados: 67
Atributos selecionados: ['Alpha_1_Antitrypsin' 'Apolipoprotein_A_IV' 'Apolipoprotein_A2'
 'Apolipoprotein_B' 'Apolipoprotein_CIII' 'Apolipoprotein_E'
 'B_Lymphocyte_Chemoattractant_BL' 'Betacellulin' 'C_Reactive_Protein'
 'Calbindin' 'Calcitonin' 'Complement_3' 'Complement_Factor_H' 'Cortisol'
 'Cystatin_C' 'EN_RAGE' 'Eotaxin_3' 'Fas_Ligand' 'Ferritin' 'Fibrinogen'
 'HB_EGF' 'I_309' 'ICAM_1' 'IL_11' 'IL_16' 'IL_17E' 'IL_4' 'IL_5' 'IL_6'
 'IL_7' 'IP_10_Inducible_Protein_10' 'Lipoprotein_a' 'MCP_2' 'MIP_1alpha'
 'MMP_2' 'MMP_3' 'MMP10' 'MMP7' 'NT_proBNP' 'NrCAM' 'Osteopontin' 'PAI_1'
 'PYY' 'Pancreatic_polypeptide' 'Pulmonary_and_Activation_Regulat'
 'RANTES' 'S100b' 'SHBG' 'SOD' 'Serum_Amyloid_P' 'Sortilin'
 'Stem_Cell_Factor' 'TGF_alpha' 'TIMP_1' 'TNF_RII' 'Thrombopoietin'
 'Thyroxine_Binding_Globulin' 'Tissue_Factor' 'VCAM_1' 'VEGF'
 'von_Willebrand_Factor' 'tau' 'p_tau' 'Ab_42' 'male' 'Genotype_E3E3'
 'Genotype_E3

## Algumas ponderações a se fazer acerca dos atributos selecionados:
Observando-os, podemos ver que alguns atributos facilmente identificáveis foram selecionados para nosso modelo.
Segundo nosso algoritmo, o fato de o paciente ser homem ou não influencia no resultado.
A presença do Cortisol também foi considerada relevante, isso é interessante pois o Cortisol é o hormônio associado ao estresse.
A ferritina, associada à disponibilidade de ferro no organismo também foi considerada relevante.
Os genótipos E3E3 e E3E4, associados ao risco de Alzheimer pela literatura também foi considerado relevante.

Vamos guardar estas conclusões e analisar estes dados isoladamente mais tarde.

In [31]:
# Exportando os dados selecionados
df_features_train_selected = df_feature_selection.loc[:, seletor.get_feature_names_out()]
df_features_train_selected.to_csv("Features selecionadas para treino.csv")
df_features_test_selected = df_features_test.loc[:, seletor.get_feature_names_out()]
df_features_test_selected.to_csv("Features selecionadas para teste.csv")